# Imports

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt

import warnings
warnings.filterwarnings("ignore")

# Wrangle

Title: IBM HR Analytics Employees Attrition & Performance

Acquired from Kaggle

Data is Synthetic and was Constructed by Data Scientists at IBM

In [2]:
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# Problem Statement

* A company has developed a new attrition prevention initiative in an attempt to save money by eliminating or reducing the cost of replacing employees. 

* The cost of implementing the initiative is 4K dollars per employee.

* The company will save 15K dollars for each employee it prevents from attriting.

* If the company were to implement this initiative for every employee in the training data set it would save about 3M. Assuming the initiative was 100 percent successful at preventing attrition.

* They would also spend about 5M implementing the initiative.

* The result is a net loss of about 2M dollars.

* This suggests that a companywide implementation of this initiative would not profitable.

* How can this company implement the new attrition initiative in a way that is cost effective?

* The cost to replace an employee was estimated using 20% the median annual employee salary from the column 'MonthlyIncome,' and is based on the findings of “There Are Significant Business Costs to Replacing Employees”, published by the Center for American Progress
* The cost of implementation is a construct of this scenario and represents an estimate of what a company might be willing to spend to prevent attrition in light of the money it will save if its efforts are successful.

# Answer

* Build a model to predict attrition. 
* Use those predictions to decide which employees to include in the initiative.
* Employees who are not going to attrit will be not be included in the initiative and will not contribute to its cost. 
* employees who are going to attrite will be included in the initiative which will cost the company about 4K dollars and (if the initiative is successful) save the company about 15K dollars, for a net gain of about 11K dollars.

# Goal

* Build a model, using DataRobot, to predict attrition in company employees

* Models will be evaluated using the max return on investment (ROI) that results from their predictions 

* A model will be considered successful if its predictions result in a positive maximum ROI for the company on holdout data


# Calculating Max ROI

* Assign a value to each prediction result (TP, TN, FN, FP)
* Use model to make predictions
* Multiply the value of each prediction result by the number of times that prediction result occurred
* Add those values together

# Prediction Result Values

##### True Positive
* Employee would have attrit and the model prediction is correct
* Employee is included in the initiative (-4k) and is presumed not to attrite as a result (+15K)
* Net Value: +11K

##### True Negative
* Employee would not have attrit and the model prediction is correct
* Employee is excluded from the initiative
* Net Value: 0

##### False Positive
* Employee would not have attrit and the model prediction is not correct
* Employee is included in the initiative (-4k) but no value is gained because the employee would not have attrite in the first place
* Net Value: -4K

##### False Negative
* Employee would have attrit and the model prediction is not correct
* Employee is not included in the initiative and attrits. The company must pay the cost or replacing that employee (-15K)
* Net Value: -15K

# Prepare
* Dropped non-informative columns
* Changed ‘survey-like’ features from numeric to categorical variables
* Removed outliers


In [37]:
df.shape

(1451, 31)

In [38]:
df.Attrition.value_counts()

No     1218
Yes     233
Name: Attrition, dtype: int64

In [39]:
for column in df.columns:
    
    print(column)
    print(df[f'{column}'].value_counts())
    print('')

Age
35    78
34    77
31    69
36    69
29    68
32    61
30    60
33    58
38    58
40    57
37    50
27    48
28    48
42    46
39    42
45    41
41    40
26    39
46    33
44    33
43    32
24    26
50    26
25    26
47    24
49    23
48    19
55    19
53    18
54    18
51    17
22    16
23    14
52    14
56    14
21    13
58    11
20    11
59    10
19     9
18     8
57     4
60     4
Name: Age, dtype: int64

Attrition
No     1218
Yes     233
Name: Attrition, dtype: int64

BusinessTravel
Travel_Rarely        1031
Travel_Frequently     273
Non-Travel            147
Name: BusinessTravel, dtype: int64

DailyRate
691     6
1329    5
329     5
408     5
1082    5
       ..
898     1
897     1
891     1
889     1
102     1
Name: DailyRate, Length: 881, dtype: int64

Department
Research & Development    950
Sales                     440
Human Resources            61
Name: Department, dtype: int64

DistanceFromHome
1     206
2     206
10     85
9      84
7      83
3      82
8      79
4     

## Dropped non-informative columns

**Contains no useful information**<br/>
EmployeeNumber

**Value is the same for all employees**<br/>
EmployeeCount<br/>
Over18<br/>
StandardHours<br/>

In [6]:
for column in df.columns:
    if column in ('EmployeeCount','EmployeeNumber','Over18','StandardHours'):
        df = df.drop(columns = f'{column}')

In [40]:
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

In [7]:
df.shape

(1470, 31)

## Changed the following ‘survey-like' features from numeric to categorical variables 

* Education

* EnvironmentSatisfaction

* JobInvolvement

* JobSatisFaction

* PerformanceRating

* RelationshipSatisfaction

* WorkLifeBalance

* JobLevel

* StockOptionLevel

In [9]:
df.Education = df.Education.apply(lambda number : str(number).replace('1','no_college'))
df.Education = df.Education.apply(lambda number : str(number).replace('2','some_college'))
df.Education = df.Education.apply(lambda number : str(number).replace('3','bachelors_degree'))
df.Education = df.Education.apply(lambda number : str(number).replace('4','masters_degree'))
df.Education = df.Education.apply(lambda number : str(number).replace('5','Doctorate'))

In [10]:
df.Education.value_counts()

bachelors_degree    572
masters_degree      398
some_college        282
no_college          170
Doctorate            48
Name: Education, dtype: int64

In [11]:
df.EnvironmentSatisfaction = df.EnvironmentSatisfaction.apply(lambda number : str(number).replace('1','low'))
df.EnvironmentSatisfaction = df.EnvironmentSatisfaction.apply(lambda number : str(number).replace('2','meduim'))
df.EnvironmentSatisfaction = df.EnvironmentSatisfaction.apply(lambda number : str(number).replace('3','high'))
df.EnvironmentSatisfaction = df.EnvironmentSatisfaction.apply(lambda number : str(number).replace('4','very_high'))

In [12]:
df.EnvironmentSatisfaction.value_counts()

high         453
very_high    446
meduim       287
low          284
Name: EnvironmentSatisfaction, dtype: int64

In [13]:
df.JobInvolvement = df.JobInvolvement.apply(lambda number : str(number).replace('1','low'))
df.JobInvolvement = df.JobInvolvement.apply(lambda number : str(number).replace('2','meduim'))
df.JobInvolvement = df.JobInvolvement.apply(lambda number : str(number).replace('3','high'))
df.JobInvolvement = df.JobInvolvement.apply(lambda number : str(number).replace('4','very_high'))

In [14]:
df.JobInvolvement.value_counts()

high         868
meduim       375
very_high    144
low           83
Name: JobInvolvement, dtype: int64

In [15]:
df.JobSatisfaction = df.JobSatisfaction.apply(lambda number : str(number).replace('1','low'))
df.JobSatisfaction = df.JobSatisfaction.apply(lambda number : str(number).replace('2','meduim'))
df.JobSatisfaction = df.JobSatisfaction.apply(lambda number : str(number).replace('3','high'))
df.JobSatisfaction = df.JobSatisfaction.apply(lambda number : str(number).replace('4','very_high'))

In [16]:
df.JobSatisfaction.value_counts()

very_high    459
high         442
low          289
meduim       280
Name: JobSatisfaction, dtype: int64

In [17]:
df.PerformanceRating = df.PerformanceRating.apply(lambda number : str(number).replace('1','low'))
df.PerformanceRating = df.PerformanceRating.apply(lambda number : str(number).replace('2','good'))
df.PerformanceRating = df.PerformanceRating.apply(lambda number : str(number).replace('3','Excellent'))
df.PerformanceRating = df.PerformanceRating.apply(lambda number : str(number).replace('4','Outstanding'))

In [18]:
df.PerformanceRating.value_counts()

Excellent      1244
Outstanding     226
Name: PerformanceRating, dtype: int64

In [19]:
df.RelationshipSatisfaction = df.RelationshipSatisfaction.apply(lambda number : str(number).replace('1','low'))
df.RelationshipSatisfaction = df.RelationshipSatisfaction.apply(lambda number : str(number).replace('2','meduim'))
df.RelationshipSatisfaction = df.RelationshipSatisfaction.apply(lambda number : str(number).replace('3','high'))
df.RelationshipSatisfaction = df.RelationshipSatisfaction.apply(lambda number : str(number).replace('4','very_high'))

In [20]:
df.RelationshipSatisfaction.value_counts()

high         459
very_high    432
meduim       303
low          276
Name: RelationshipSatisfaction, dtype: int64

In [21]:
df.WorkLifeBalance = df.WorkLifeBalance.apply(lambda number : str(number).replace('1','bad'))
df.WorkLifeBalance = df.WorkLifeBalance.apply(lambda number : str(number).replace('2','good'))
df.WorkLifeBalance = df.WorkLifeBalance.apply(lambda number : str(number).replace('3','better'))
df.WorkLifeBalance = df.WorkLifeBalance.apply(lambda number : str(number).replace('4','best'))

In [22]:
df.WorkLifeBalance.value_counts()

better    893
good      344
best      153
bad        80
Name: WorkLifeBalance, dtype: int64

In [23]:
df.JobLevel = df.JobLevel.apply(lambda number : str(number).replace('1','one'))
df.JobLevel = df.JobLevel.apply(lambda number : str(number).replace('2','two'))
df.JobLevel = df.JobLevel.apply(lambda number : str(number).replace('3','three'))
df.JobLevel = df.JobLevel.apply(lambda number : str(number).replace('4','four'))
df.JobLevel = df.JobLevel.apply(lambda number : str(number).replace('5','five'))

In [24]:
df.JobLevel.value_counts()

one      543
two      534
three    218
four     106
five      69
Name: JobLevel, dtype: int64

In [25]:
df.StockOptionLevel = df.StockOptionLevel.apply(lambda number : str(number).replace('0','zero'))
df.StockOptionLevel = df.StockOptionLevel.apply(lambda number : str(number).replace('1','one'))
df.StockOptionLevel = df.StockOptionLevel.apply(lambda number : str(number).replace('2','two'))
df.StockOptionLevel = df.StockOptionLevel.apply(lambda number : str(number).replace('3','three'))

In [26]:
df.StockOptionLevel.value_counts()

zero     631
one      596
two      158
three     85
Name: StockOptionLevel, dtype: int64

## Removing Outliers 
* After puting the data with the above transformations into DataRobot it identified the 'YearsAtCompany' column as containing outliers
* I reviewed the data I decided to drop outliers using the interquortile rule
* This removed employees with a 'YearsAtCompany' value greater than 27 from the dataset
* Dropped lines totaled 19

In [28]:
def get_outliers_iqr(df, col):
    
    q75, q25 = np.percentile(df[col], [75,25])
    upper_bound = 3*stats.iqr(df[col]) + q75
    lower_bound = q25 - 3*stats.iqr(df[col])
        
    return upper_bound, lower_bound

In [29]:
get_outliers_iqr(df, ['YearsAtCompany'])

(27.0, -15.0)

In [30]:
def remove_outliers_iqr(df, columns):
    for col in columns:
        q75, q25 = np.percentile(df[col], [75,25])
        ub = 3*stats.iqr(df[col]) + q75
        lb = q25 - 3*stats.iqr(df[col])
        df = df[df[col] <= ub]
        df = df[df[col] >= lb]
    return df

In [31]:
df = remove_outliers_iqr(df, ['YearsAtCompany'])

In [33]:
employee_churn_outliers_removed = df

employee_churn_outliers_removed.YearsAtCompany.max()

27

In [34]:
employee_churn_outliers_removed.shape

(1451, 31)

# Create CSV with Prepared Data
* The CSV was then used to explore the data and create models in DataRobot

In [ ]:
#employee_churn_outliers_removed.to_csv('employee_churn_outliers_removed.csv',index=False)

# Feature Engineering
* Values with similar rates of Attrition within a given categorical feature were merged together in ‘value clusters’
* When Datarobot creates models, it encodes each value in categorical features as separate feature
* Creating value clusters would reduce the number of post-encoding features that would go into each model while preserving most of the information contained in each of the pre-encoding features

**The following features were created using 'value clusters’ of a parent feature. The parent feature was then dropped from the dataset.**
* JobRoleCluster
* StockOptionLevelCluster
* JobLevelCluster
* JobSatisfactionCluster
* EnvironmentSatisfactionCluster
* EducationFieldCluster
* RelationshipSatisfactionCluster
* MaritalStatusCluster
* WorkLifeBalanceCluster
* JobInvolvementCluster



In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.JobRole.value_counts()

In [ ]:
df['JobRoleCluster'] = df.JobRole

In [ ]:
df.JobRoleCluster.value_counts()

In [ ]:
df.JobRole.value_counts()

In [ ]:
df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Sales Executive','Cluster 1'))
df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Research Scientist','Cluster 1'))

df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Laboratory Technician','Cluster 2'))
df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Human Resources','Cluster 2'))

df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Manufacturing Director','Cluster 3'))
df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Healthcare Representative','Cluster 3'))
df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Manager','Cluster 3'))
df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Research Director','Cluster 3'))

df.JobRoleCluster = df.JobRoleCluster.apply(lambda value : value.replace('Sales Representative','Cluster 4'))

df.JobRoleCluster.value_counts()

In [ ]:
df.StockOptionLevel.value_counts()

In [ ]:
df['StockOptionLevelCluster'] = df.StockOptionLevel

df.StockOptionLevelCluster = df.StockOptionLevelCluster.apply(lambda value : value.replace('zero','Cluster 1'))
df.StockOptionLevelCluster = df.StockOptionLevelCluster.apply(lambda value : value.replace('one','Cluster 2'))
df.StockOptionLevelCluster = df.StockOptionLevelCluster.apply(lambda value : value.replace('two','Cluster 2'))
df.StockOptionLevelCluster = df.StockOptionLevelCluster.apply(lambda value : value.replace('three','Cluster 3'))


df.StockOptionLevelCluster.value_counts()

In [ ]:
df.JobLevel.value_counts()

In [ ]:
df['JobLevelCluster'] = df.JobLevel

df.JobLevelCluster = df.JobLevelCluster.apply(lambda value : value.replace('one','Cluster 1'))
df.JobLevelCluster = df.JobLevelCluster.apply(lambda value : value.replace('three','Cluster 2'))
df.JobLevelCluster = df.JobLevelCluster.apply(lambda value : value.replace('two','Cluster 3'))
df.JobLevelCluster = df.JobLevelCluster.apply(lambda value : value.replace('four','Cluster 3'))
df.JobLevelCluster = df.JobLevelCluster.apply(lambda value : value.replace('five','Cluster 3'))

df['JobLevelCluster'].value_counts()

In [ ]:
df.JobSatisfaction.value_counts()

In [ ]:
df['JobSatisfactionCluster'] = df.JobSatisfaction

df.JobSatisfactionCluster = df.JobSatisfactionCluster.apply(lambda value : value.replace('very_high','Cluster 1'))
df.JobSatisfactionCluster = df.JobSatisfactionCluster.apply(lambda value : value.replace('high','Cluster 2'))
df.JobSatisfactionCluster = df.JobSatisfactionCluster.apply(lambda value : value.replace('meduim','Cluster 2'))
df.JobSatisfactionCluster = df.JobSatisfactionCluster.apply(lambda value : value.replace('low','Cluster 3'))


df['JobSatisfactionCluster'].value_counts()

In [ ]:
df.EnvironmentSatisfaction.value_counts()

In [ ]:
df['EnvironmentSatisfactionCluster'] = df.EnvironmentSatisfaction

df.EnvironmentSatisfactionCluster = df.EnvironmentSatisfactionCluster.apply(lambda value : value.replace('very_high','Cluster 1'))
df.EnvironmentSatisfactionCluster = df.EnvironmentSatisfactionCluster.apply(lambda value : value.replace('high','Cluster 2'))
df.EnvironmentSatisfactionCluster = df.EnvironmentSatisfactionCluster.apply(lambda value : value.replace('meduim','Cluster 2'))
df.EnvironmentSatisfactionCluster = df.EnvironmentSatisfactionCluster.apply(lambda value : value.replace('low','Cluster 2'))


df['EnvironmentSatisfactionCluster'].value_counts()

In [ ]:
df.EducationField.value_counts()

In [ ]:
df['EducationFieldCluster'] = df.EducationField

df.EducationFieldCluster = df.EducationFieldCluster.apply(lambda value : value.replace('Life Sciences','Cluster 1'))
df.EducationFieldCluster = df.EducationFieldCluster.apply(lambda value : value.replace('Medical','Cluster 1'))
df.EducationFieldCluster = df.EducationFieldCluster.apply(lambda value : value.replace('Other','Cluster 1'))
df.EducationFieldCluster = df.EducationFieldCluster.apply(lambda value : value.replace('Marketing','Cluster 2'))
df.EducationFieldCluster = df.EducationFieldCluster.apply(lambda value : value.replace('Technical Degree','Cluster 2'))
df.EducationFieldCluster = df.EducationFieldCluster.apply(lambda value : value.replace('Human Resources','Cluster 3'))


df['EducationFieldCluster'].value_counts()

In [ ]:
df.RelationshipSatisfaction.value_counts()

In [ ]:
df['RelationshipSatisfactionCluster'] = df.RelationshipSatisfaction

df.RelationshipSatisfactionCluster = df.RelationshipSatisfactionCluster.apply(lambda value : value.replace('very_high','Cluster 1'))
df.RelationshipSatisfactionCluster = df.RelationshipSatisfactionCluster.apply(lambda value : value.replace('high','Cluster 1'))
df.RelationshipSatisfactionCluster = df.RelationshipSatisfactionCluster.apply(lambda value : value.replace('meduim','Cluster 1'))
df.RelationshipSatisfactionCluster = df.RelationshipSatisfactionCluster.apply(lambda value : value.replace('low','Cluster 2'))


df['RelationshipSatisfactionCluster'].value_counts()

In [ ]:
df.MaritalStatus.value_counts()

In [ ]:
df['MaritalStatusCluster'] = df.MaritalStatus

df.MaritalStatusCluster = df.MaritalStatusCluster.apply(lambda value : value.replace('Married','Cluster 1'))
df.MaritalStatusCluster = df.MaritalStatusCluster.apply(lambda value : value.replace('Divorced','Cluster 1'))
df.MaritalStatusCluster = df.MaritalStatusCluster.apply(lambda value : value.replace('Single','Cluster 2'))



df['MaritalStatusCluster'].value_counts()

In [ ]:
df.WorkLifeBalance.value_counts()

In [ ]:
df['WorkLifeBalanceCluster'] = df.WorkLifeBalance

df.WorkLifeBalanceCluster = df.WorkLifeBalanceCluster.apply(lambda value : value.replace('best','Cluster 1'))
df.WorkLifeBalanceCluster = df.WorkLifeBalanceCluster.apply(lambda value : value.replace('better','Cluster 1'))
df.WorkLifeBalanceCluster = df.WorkLifeBalanceCluster.apply(lambda value : value.replace('good','Cluster 1'))
df.WorkLifeBalanceCluster = df.WorkLifeBalanceCluster.apply(lambda value : value.replace('bad','Cluster 2'))

df['WorkLifeBalanceCluster'].value_counts()

In [ ]:
df.JobInvolvement.value_counts()

In [ ]:
df['JobInvolvementCluster'] = df.JobInvolvement

df.JobInvolvementCluster = df.JobInvolvementCluster.apply(lambda value : value.replace('very_high','Cluster 1'))
df.JobInvolvementCluster = df.JobInvolvementCluster.apply(lambda value : value.replace('high','Cluster 1'))
df.JobInvolvementCluster = df.JobInvolvementCluster.apply(lambda value : value.replace('meduim','Cluster 2'))
df.JobInvolvementCluster = df.JobInvolvementCluster.apply(lambda value : value.replace('low','Cluster 3'))

df['JobInvolvementCluster'].value_counts()

In [ ]:
df = df.drop(columns=[ 'EducationField','EnvironmentSatisfaction','JobInvolvement',
                  'JobLevel', 'JobRole', 'JobSatisfaction', 
                  'MaritalStatus','RelationshipSatisfaction','StockOptionLevel', 
                  'WorkLifeBalance','EducationField'])

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
#df.to_csv('employee_churn_catagorical_clusters.csv',index=False)